In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
from os import listdir
from os.path import isfile, join

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
import calendar

In [4]:
idx = pd.IndexSlice

# IT_Op_w_Products

In [15]:
op_w_prod = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Op_w_Products.csv", encoding='latin-1', sep=";")

for col in forecast.columns:
    if "Date" in col:
        forecast[col] = forecast[col].astype(str)
        forecast[col] = pd.to_datetime(forecast[col], format="%d/%m/%Y")
    else:
        pass


columns = [c for c in forecast.columns if not "Currency" in c]
forecast = forecast.loc[:,columns]

num = []

for col in forecast.columns:
    if "Amount" in col:
        num.append(col)
    else:
        pass
    
forecast[num] = forecast[num].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

In [ ]:
op_w_prod["Product Name"].fillna("-", inplace=True)
op_w_prod["Products"] = op_w_prod.groupby("Opportunity ID")["Product Name"].transform(lambda x: " | ".join(x))

In [16]:
op_w_prod["Opp. ID freq"] = op_w_prod.groupby("Opportunity ID")["Opportunity ID"].transform("count")
op_w_prod["Opp. Incr. Amount"] = op_w_prod["Incremental Amount"] / op_w_prod["Opp. ID freq"]

In [17]:
op_w_prod["Stage cat"] = pd.Categorical(op_w_prod["Stage"], ["Qualification","Needs Analysis","Solution Presentation / Demo", "Proposal", "Negotiation","Closed Won", "Closed Lost"])

In [18]:
op_w_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25669 entries, 0 to 25668
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Opportunity ID           25669 non-null  object        
 1   Opportunity Name         25669 non-null  object        
 2   Owner Role               25669 non-null  object        
 3   Opportunity Owner        25669 non-null  object        
 4   Probability (%)          25669 non-null  int64         
 5   Stage                    25669 non-null  object        
 6   Created Date             25669 non-null  datetime64[ns]
 7   Close Date               25669 non-null  datetime64[ns]
 8   Age                      25669 non-null  int64         
 9   Stage Duration           25669 non-null  int64         
 10  Subscription Amount      25664 non-null  float64       
 11  Non-Subscription Amount  25632 non-null  float64       
 12  Incremental Amount       24798 n

## Opportunity by stage

### Opportunity by stage by owner role

In [19]:
owner_role = ['IT North West Sales Team', 'IT North East Sales Team', 'IT Enterprise Specialists Sales Team', 'IT Central & South Sales Team', 'IT Large Account Sales Team']
filt = op_w_prod["Owner Role"].isin(owner_role)

op_by_stage_owner_role = op_w_prod.loc[filt,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat"]).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity ID":"nunique", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity ID":"N. Opportunità"})

op_by_stage_owner_role.index.rename(("Owner Role","Stage"), inplace=True)

In [20]:
op_by_stage_owner_role

Stage Duration  \
Owner Role                           Stage                                          
IT Central & South Sales Team        Qualification                     147.921708   
                                     Needs Analysis                      5.500000   
                                     Solution Presentation / Demo      193.006154   
                                     Proposal                          203.204152   
                                     Negotiation                       351.666667   
                                     Closed Won                        281.326112   
                                     Closed Lost                       282.182648   
IT Enterprise Specialists Sales Team Qualification                      87.273810   
                                     Needs Analysis                      7.000000   
                                     Solution Presentation / Demo      105.428571   
                                     Proposal                          154.738462   
                                     Negotiation                       105.750000   
                                     Closed Won                        288.496503   
                                     Closed Lost                       276.356959   
IT Large Account Sales Team          Qualification                     237.481481   
                                     Needs Analysis                    197.000000   
                                     Solution Presentation / Demo      345.750000   
                                     Proposal                          427.666667   
                                     Negotiation                              NaN   
                                     Closed Won                        354.056872   
                                     Closed Lost                       342.536364   
IT North East Sales Team             Qualification                     127.495935   
                                     Needs Analysis                      4.800000   
                                     Solution Presentation / Demo      111.572864   
                                     Proposal                          126.407473   
                                     Negotiation                        57.157895   
                                     Closed Won                        282.820176   
                                     Closed Lost                       260.187196   
IT North West Sales Team             Qualification                     122.600962   
                                     Needs Analysis                     41.000000   
                                     Solution Presentation / Demo      135.864130   
                                     Proposal                          175.784530   
                                     Negotiation                        78.288889   
                                     Closed Won                        283.891513   
                                     Closed Lost                       238.144700   

                                                                          Age  \
Owner Role                           Stage                                      
IT Central & South Sales Team        Qualification                 154.729537   
                                     Needs Analysis                  5.500000   
                                     Solution Presentation / Demo  215.406154   
                                     Proposal                      209.027682   
                                     Negotiation                   374.274510   
                                     Closed Won                     20.180160   
                                     Closed Lost                   103.601218   
IT Enterprise Specialists Sales Team Qualification                  90.571429   
                                     Needs Analysis                 27.000000   
                              

### Opportunity by stage

In [21]:
op_by_stage = op_w_prod.sort_values(['Stage cat']).groupby(["Stage cat"]).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity ID":"nunique", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity ID":"N. Opportunità"})

op_by_stage.index.rename(("Stage"), inplace=True)

In [22]:
op_by_stage

,Stage Duration,Age,N. Opportunità,Opp. Incr. Amount,Total Price
Stage,,,,,
Qualification,133.460581,139.550484,682,1045370.33,1045370.33
Needs Analysis,59.000000,64.733333,13,151768.00,151768.00
Solution Presentation / Demo,150.772040,169.803526,614,1194772.45,1194772.45
Proposal,161.590847,174.060792,1049,1178263.90,1178263.93
Negotiation,166.467836,181.315789,113,223612.00,223612.00
Closed Won,284.175601,17.487694,12394,14062955.89,14062955.95
Closed Lost,259.897679,110.368929,4033,5813449.75,5813449.85


## Velocity

#### Velocity by stage by owner role - Opportunità aperte

In [23]:
velocity_stage_owner_role_open = op_by_stage_owner_role.loc[idx[:, ["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["Stage Duration", "N. Opportunità"]]

velocity_stage_owner_role_open.style.format("{:,.0f}")


#### Velocity by stage by owner role - Opportunità chiuse

In [24]:
velocity_stage_owner_role_closed = op_by_stage_owner_role.loc[idx[:, ["Closed Won", "Closed Lost"]], idx["Age", "N. Opportunità"]]

velocity_stage_owner_role_closed.style.format("{:,.0f}")


#### Velocity by stage - Opportunità aperte

In [25]:
velocity_stage_open = op_by_stage.loc[idx[["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["Stage Duration", "N. Opportunità"]]

velocity_stage_open.style.format("{:,.0f}").background_gradient()


,Stage Duration,N. Opportunità
Stage,,
Qualification,133,682
Needs Analysis,59,13
Solution Presentation / Demo,151,614
Proposal,162,"1,049"
Negotiation,166,113


#### Velocity by stage - Opportunità chiuse

In [26]:
velocity_stage_closed = op_by_stage.loc[idx[ ["Closed Won", "Closed Lost"]], idx["Age", "N. Opportunità"]]

velocity_stage_closed.style.format("{:,.0f}").background_gradient()


,Age,N. Opportunità
Stage,,
Closed Won,17,"12,394"
Closed Lost,110,"4,033"


## Volume

#### Volume by stage by owner role

In [27]:
volume_stage_owner_role = op_by_stage_owner_role.loc[idx[:, ["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["N. Opportunità", "Opp. Incr. Amount"]]

volume_stage_owner_role.style.format("{:,.0f}")


#### Volume by stage

In [28]:
volume_stage = op_by_stage.loc[idx[["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]], idx["N. Opportunità", "Opp. Incr. Amount"]]

volume_stage.style.format("{:,.0f}")

,N. Opportunità,Opp. Incr. Amount
Stage,,
Qualification,682,"1,045,370"
Needs Analysis,13,"151,768"
Solution Presentation / Demo,614,"1,194,772"
Proposal,"1,049","1,178,264"
Negotiation,113,"223,612"


In [29]:
volume_stage_by_date = op_w_prod.sort_values(['Stage cat']).groupby(["Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"}).unstack(level=1)    

volume_stage_by_date.style.format("{:,.0f}")

In [52]:
op_w_prod["Pipeline seniority"] = op_w_prod["Created Date"].apply(lambda x: np.where(x<pd.Timestamp(dt.datetime.now().date() - dt.timedelta(days=270)), "Older than 9 month", "Younger than 9 month"))

In [57]:
volume_stage_by_seniority = op_w_prod.sort_values(['Stage cat']).groupby(["Stage cat","Pipeline seniority"]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"}).unstack(level=1).loc[idx["Qualification":"Negotiation"]]

volume_stage_by_seniority.style.format("{:,.0f}").background_gradient()   

## Export to excel

In [58]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage.xlsx", engine="openpyxl") as writer:
    velocity_stage_open.to_excel(writer, sheet_name="vel_open")
    velocity_stage_owner_role_open.to_excel(writer, sheet_name="vel_open_dett")
    velocity_stage_closed.to_excel(writer, sheet_name="vel_closed")
    velocity_stage_owner_role_closed.to_excel(writer, sheet_name="vel_closed_dett")
    volume_stage.to_excel(writer, sheet_name="vol_open")
    volume_stage_owner_role.to_excel(writer, sheet_name="vol_open_dett")
    volume_stage_by_date.to_excel(writer, sheet_name="vol_by_date")
    volume_stage_by_seniority.to_excel(writer, sheet_name="vol_by_seniority")

---

### N. of opportunity by stage and created date and FILTERED by stage duration limits

In [ ]:
qualification = op_w_prod.loc[(op_w_prod["Stage cat"] == "Qualification") & (op_w_prod["Stage Duration"] > 7),:]
analysis = op_w_prod.loc[(op_w_prod["Stage cat"] == "Needs Analysis") & (op_w_prod["Stage Duration"] > 15),:]
demo = op_w_prod.loc[(op_w_prod["Stage cat"] == "Solution Presentation / Demo") & (op_w_prod["Stage Duration"] > 20) & (op_w_prod["Close Date"].dt.month == dt.datetime.now().month),:]
proposal = op_w_prod.loc[(op_w_prod["Stage cat"] == "Proposal") & (op_w_prod["Stage Duration"] > 15),:]
negotiation = op_w_prod.loc[(op_w_prod["Stage cat"] == "Negotiation") & (op_w_prod["Stage Duration"] > 30),:]

op_w_prod_filt = pd.concat([qualification,analysis,demo,proposal,negotiation], axis=0)

op_w_prod_filt_gr = op_w_prod_filt.sort_values(['Stage cat']).groupby(["Owner Role","Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"}).unstack(level=2)

op_w_prod_filt_sintesi = op_w_prod_filt.sort_values(['Stage cat']).groupby(["Owner Role", "Stage cat"]).agg({"Opportunity ID":"nunique", "Opp. Incr. Amount":"sum"})


op_w_prod_filt_gr.loc[idx[:,["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]],:]

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_date_and_stage_duration.xlsx", engine="openpyxl") as writer:
    qualification.to_excel(writer, sheet_name="qualification")
    analysis.to_excel(writer, sheet_name="analysis")
    demo.to_excel(writer, sheet_name="demo")
    proposal.to_excel(writer, sheet_name="proposal")
    negotiation.to_excel(writer, sheet_name="negotiation")
    op_w_prod_filt_gr.loc[idx[:,["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]],:].to_excel(writer, sheet_name="op_w_prod_filt_gr")
    op_w_prod_filt_sintesi.loc[idx[:,["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]],:].to_excel(writer, sheet_name="op_w_prod_filt_sintesi")

### Opportunity by stage and created date

In [32]:
op_w_prod_sintesi = op_w_prod.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat",op_w_prod["Created Date"].astype("datetime64[M]")]).agg({"Opp. Incr. Amount":"sum"}).unstack(level=2)

op_w_prod_sintesi.index.rename(("Owner Role","Stage"), inplace=True)

#op_w_prod_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_date.xlsx")
op_w_prod_sintesi.loc[idx[:,["Qualification","Needs Analysis", "Solution Presentation / Demo", "Proposal", "Negotiation"]],]

Opp. Incr. Amount  \
Created Date                                                             2020-10-01   
Owner Role                           Stage                                            
IT Central & South Manager           Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo               0.0   
                                     Proposal                                   0.0   
                                     Negotiation                                0.0   
IT Central & South Sales Team        Qualification                           4245.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo             500.0   
                                     Proposal                                1100.0   
                                     Negotiation                             6000.0   
IT Enterprise Specialists Sales Team Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo               0.0   
                                     Proposal                               20390.0   
                                     Negotiation                                0.0   
IT Large Account Sales Team          Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo               0.0   
                                     Proposal                                   0.0   
                                     Negotiation                                0.0   
IT Large Accounts                    Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo               0.0   
                                     Proposal                                   0.0   
                                     Negotiation                                0.0   
IT North East Sales Team             Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo            2310.0   
                                     Proposal                                4380.0   
                                     Negotiation                                0.0   
IT North West Sales Team             Qualification                              0.0   
                                     Needs Analysis                             0.0   
                                     Solution Presentation / Demo               0.0   
                                     Proposal                               18610.0   
                                     Negotiation                                0.0   

                                                                              \
Created Date                                                      2020-11-01   
Owner Role                           Stage                                     
IT Central & South Manager           Qualification                       0.0   
                                     Needs Analysis                      0.0   
                                     Solution Presentation / Demo        0.0   
                                     Proposal                            0.0   
                                     Negotiation                         0.0   
IT Central & South Sales Team        Qualification                    5045.0   
                                     Ne

### Opportunity by stage and close date

In [36]:
op_w_prod_sintesi = op_w_prod.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","Stage cat",op_w_prod["Close Date"].astype("datetime64[M]")]).agg({"Opp. Incr. Amount":"sum"}).unstack(level=2)

op_w_prod_sintesi.index.rename(("Owner Role","Stage"), inplace=True)

#op_w_prod_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_close_date.xlsx")
op_w_prod_sintesi.loc[idx[:,["Closed Lost","Closed Won"]],]

Opp. Incr. Amount             \
Close Date                                              2020-10-01 2020-11-01   
Owner Role                           Stage                                      
IT Central & South Manager           Closed Lost               0.0    2960.00   
IT Central & South Sales Team        Closed Lost               0.0   41560.00   
IT Enterprise Specialists Sales Team Closed Lost            3040.0   19935.00   
IT Large Account Sales Team          Closed Lost               0.0   11600.00   
IT Large Accounts                    Closed Lost               0.0       0.00   
IT North East Sales Team             Closed Lost            1790.0   28202.00   
IT North West Sales Team             Closed Lost            1420.0    9722.00   
IT Central & South Manager           Closed Won                0.0       0.00   
IT Central & South Sales Team        Closed Won            10000.0  118397.99   
IT Enterprise Specialists Sales Team Closed Won            14204.0  136135.50   
IT Large Account Sales Team          Closed Won             2650.0  112993.95   
IT Large Accounts                    Closed Won                0.0  116600.00   
IT North East Sales Team             Closed Won            55897.0  169221.10   
IT North West Sales Team             Closed Won            52544.0  222152.00   

                                                                        \
Close Date                                       2020-12-01 2021-01-01   
Owner Role                           Stage                               
IT Central & South Manager           Closed Lost       0.00    3200.00   
IT Central & South Sales Team        Closed Lost   22524.80   36720.00   
IT Enterprise Specialists Sales Team Closed Lost   49070.00   22480.00   
IT Large Account Sales Team          Closed Lost   81045.00    9350.00   
IT Large Accounts                    Closed Lost    9000.00       0.00   
IT North East Sales Team             Closed Lost   48021.00   42401.00   
IT North West Sales Team             Closed Lost   35870.42   35424.00   
IT Central & South Manager           Closed Won        0.00       0.00   
IT Central & South Sales Team        Closed Won   114057.00  128894.72   
IT Enterprise Specialists Sales Team Closed Won   128078.00  109983.50   
IT Large Account Sales Team          Closed Won   138290.00   36384.00   
IT Large Accounts                    Closed Won        0.00   35500.00   
IT North East Sales Team             Closed Won   133089.00  110039.50   
IT North West Sales Team             Closed Won   140264.00  158245.50   

                                                                        \
Close Date                                       2021-02-01 2021-03-01   
Owner Role                           Stage                               
IT Central & South Manager           Closed Lost       0.00       0.00   
IT Central & South Sales Team        Closed Lost   60510.00   38350.00   
IT Enterprise Specialists Sales Team Closed Lost   22490.00   57151.00   
IT Large Account Sales Team          Closed Lost     800.00  128625.00   
IT Large Accounts                    Closed Lost       0.00       0.00   
IT North East Sales Team             Closed Lost   80209.00   83283.84   
IT North West Sales Team             Closed Lost   48480.00   32182.58   
IT Central & South Manager           Closed Won        0.00       0.00   
IT Central & South Sales Team        Closed Won   123373.49  138656.72   
IT Enterprise Specialists Sales Team Closed Won   146932.00  151213.00   
IT Large Account Sales Team          Closed Won    36990.00  147887.44   
IT Large Accounts                    Closed Won        0.00   51000.00   
IT North East Sales Team             Closed Won   185906.72  160890.15   
IT North West Sales Team             Closed Won   175893.83  240055.62   

                                                                        \
Close Date                                       2021-04-01 2021-05-01   
Owner

### Opportunity by stage and product

In [ ]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [ ]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod.loc[:, "Codice PRODOTTO"].apply(lambda x: "IT-" + x.zfill(8))

In [ ]:
op_w_prod_merged = op_w_prod.merge(cod_prod, how="left", left_on="Product Code", right_on="Codice PRODOTTO")

In [ ]:
op_w_prod_merged_sintesi = op_w_prod_merged.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","MDM","Stage cat"], dropna=False).agg({"Stage Duration":"mean", "Age":"mean", "Opportunity Name":"count", "Opp. Incr. Amount":"sum", "Total Price":"sum"}).rename(columns={"Opportunity Name":"N. Opportunità"})

op_w_prod_merged_sintesi.index.rename(("Owner Role","Product","Stage"), inplace=True)

op_w_prod_merged_sintesi.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_prod.xlsx")

op_w_prod_merged_sintesi

In [ ]:
op_w_prod_merged_sintesi = op_w_prod_merged.loc[:,:].sort_values(['Stage cat']).groupby(["Owner Role","MDM","Stage cat"], dropna=False).agg({"Stage Duration":"mean"})
op_w_prod_merged_sintesi.index.rename(("Owner Role","Product","Stage"), inplace=True)
op_w_prod_merged_sintesi.unstack(level=1).to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Reports\opportunity_by_stage_and_prod.xlsx", )


In [ ]:
op_w_prod_merged_sintesi.unstack(level=1)